### Dummy GPT Model 
A GPT arhitecture contains embedding + positional encoding --> normalization --> some other operations in between --> transformer (scaled dot product attention in multi head attention + masked attention) 

In [1]:
import torch 
import torch.nn as nn 

### Backbone Terms: 
- vocabulary size: total # of vocabularies in the training dataset (depends on tokenization, we may add other <> tokens as well). 
- context_length: max number of input tokens taken at each time 
- embed_dim: the dimension of the word embedding at encoding stage (more dim can improve words learning better)
- n_layers: number of hidden layers in the transformer block 
- n_heads: # of attention heads 

In [ ]:
config= {
    "vocab_size" : 50257, 
    "context_length" : 1024, 
    "embed_dim" : 768,
    "n_heads" : 12, 
    "n_layers" : 12, 
    "drop_rate" : 0.1  # each node during the hidden layer has 10% being dropped. Avoids overfitting 
}

In [ ]:
class GPTModel(nn.Moduel):
    def __init__(self, cfg): 
        super().__init__() # init torch.nn.Module class 
        self.embed= nn.Embedding(cfg["vocab_size"], cfg["embed_dim"])
        self.pos_embed= nn.Embedding(cfg["context_length"], cfg["embed_dim"])   # add the positional encoding. 
        # input will be the same as the context length (# of tokens processed each time, and embed them at a hidden dimension)

        self.drop_embed= nn.Dropout(cfg["drop_rate"])   # drop out function 
        #TODO: self.transformer = nn.Sequential () # transformer block 
        #TODO: self.final_norm= LayerNorm (cfg["embed_dim"])   # layer normalization 
        self.out_head= nn.Linear(cfg["embed_dim"], cfg["vocab_size"], bias=False)   # final FC layer back to vocabulary

    # return the non-softmax output of 
    def forward(self, in_idx):
        batch_size, seq_len= in_idx.shape # vector would be importe as the size batch x context_length x vocab size 
        text_embed= self.embed(in_idx)
        pos_embed= self.pos_embed(torch.arange(seq_len, device= in_idx.device)) # non-repeating positional encoding (copies the entire weight matrix of the token_embed)
        
        # step 1: add embedding and positional encoding 
        x= pos_embed + text_embed 
        # step 2: apply drop out 
        x= self.drop_embed(x)
        # step 3 :go through the transformer 
        x= self.transformer(x) 
        # step 4: more layer normalization and then FC to output 
        x= self.final_norm(x) 
        logits= self.out_head(x) 
        return logits 
        
        

